![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/ER_SNOMED_ES.ipynb)

# Colab Setup

In [1]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [2]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.0 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 6.3.2
Spark NLP_JSL Version : 6.3.0


# Checking in an inference pipeline

In [39]:
document_assembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx")\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")\

word_embeddings = WordEmbeddingsModel.pretrained("w2v_cc_300d","es")\
  .setInputCols(["sentence","token"])\
  .setOutputCol("embeddings")

ner_eu = MedicalNerModel.pretrained("ner_eu_clinical_condition", "es", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner_eu")

ner_eu_converter = NerConverterInternal() \
  .setInputCols(["sentence", "token", "ner_eu"]) \
  .setOutputCol("ner_eu_chunk")

chunk2doc = Chunk2Doc()\
  .setInputCols("ner_eu_chunk")\
  .setOutputCol("ner_chunk_doc")

biolord_embeddings = XlmRoBertaSentenceEmbeddings.pretrained("sent_xlm_roberta_biolord_2023_m","xx")\
    .setInputCols(["ner_chunk_doc"])\
    .setOutputCol("biolord_embeddings")

snomed_resolver = SentenceEntityResolverModel.pretrained("biolordresolve_snomed_augmented","es", "clinical/models") \
      .setInputCols(["biolord_embeddings"]) \
      .setOutputCol("snomed_code")\
      .setDistanceFunction("EUCLIDEAN")

snomed_pipeline = Pipeline(stages = [
    document_assembler,
    sentence_detector,
    tokenizer,
    word_embeddings,
    ner_eu,
    ner_eu_converter,
    chunk2doc,
    biolord_embeddings,
    snomed_resolver
])

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
w2v_cc_300d download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
ner_eu_clinical_condition download started this may take some time.
Approximate size to download 877.1 KB
[OK!]
sent_xlm_roberta_biolord_2023_m download started this may take some time.
Approximate size to download 973.6 MB
[OK!]
biolordresolve_snomed_augmented download started this may take some time.
Approximate size to download 2.2 GB
[OK!]


In [40]:
import pandas as pd

clinical_note = 'Mujer de 28 años con antecedentes de diabetes mellitus gestacional diagnosticada ocho años antes de la presentación y posterior diabetes mellitus tipo dos (DM2), un episodio previo de pancreatitis inducida por HTG tres años antes de la presentación, asociado con una hepatitis aguda, y obesidad con un índice de masa corporal (IMC) de 33,5 kg / m2, que se presentó con antecedentes de una semana de poliuria, polidipsia, falta de apetito y vómitos. Dos semanas antes de la presentación, fue tratada con un ciclo de cinco días de amoxicilina por una infección del tracto respiratorio. Estaba tomando metformina, glipizida y dapagliflozina para la DM2 y atorvastatina y gemfibrozil para la HTG. Había estado tomando dapagliflozina durante seis meses en el momento de la presentación. El examen físico al momento de la presentación fue significativo para la mucosa oral seca; significativamente, su examen abdominal fue benigno sin dolor a la palpación, protección o rigidez. Los hallazgos de laboratorio pertinentes al ingreso fueron: glucosa sérica 111 mg / dl, bicarbonato 18 mmol / l, anión gap 20, creatinina 0,4 mg / dl, triglicéridos 508 mg / dl, colesterol total 122 mg / dl, hemoglobina glucosilada (HbA1c) 10%. y pH venoso 7,27. La lipasa sérica fue normal a 43 U / L. Los niveles séricos de acetona no pudieron evaluarse ya que las muestras de sangre se mantuvieron hemolizadas debido a una lipemia significativa. La paciente ingresó inicialmente por cetosis por inanición, ya que refirió una ingesta oral deficiente durante los tres días previos a la admisión. Sin embargo, la química sérica obtenida seis horas después de la presentación reveló que su glucosa era de 186 mg / dL, la brecha aniónica todavía estaba elevada a 21, el bicarbonato sérico era de 16 mmol / L, el nivel de triglicéridos alcanzó un máximo de 2050 mg / dL y la lipasa fue de 52 U / L. Se obtuvo el nivel de β-hidroxibutirato y se encontró que estaba elevado a 5,29 mmol / L; la muestra original se centrifugó y la capa de quilomicrones se eliminó antes del análisis debido a la interferencia de la turbidez causada por la lipemia nuevamente. El paciente fue tratado con un goteo de insulina para euDKA y HTG con una reducción de la brecha aniónica a 13 y triglicéridos a 1400 mg / dL, dentro de las 24 horas. Se pensó que su euDKA fue precipitada por su infección del tracto respiratorio en el contexto del uso del inhibidor de SGLT2. La paciente fue atendida por el servicio de endocrinología y fue dada de alta con 40 unidades de insulina glargina por la noche, 12 unidades de insulina lispro con las comidas y metformina 1000 mg dos veces al día. Se determinó que todos los inhibidores de SGLT2 deben suspenderse indefinidamente. Tuvo un seguimiento estrecho con endocrinología post alta.'

data = spark.createDataFrame([[clinical_note]]).toDF("text")

snomed_result = snomed_pipeline.fit(data).transform(data)

In [41]:
import pandas as pd

chunk = 'ner_eu_chunk'
output_col='snomed_code'

pd.set_option('display.max_colwidth', 0)

df = snomed_result.select(F.explode(F.arrays_zip(snomed_result[chunk].begin,
                                                 snomed_result[chunk].end,
                                                 snomed_result[chunk].result,
                                                 snomed_result[chunk].metadata,
                                                 snomed_result[output_col].result,
                                                 snomed_result[output_col].metadata)).alias("cols")) \
              .select(F.expr("cols['0']").alias("begin"),
                      F.expr("cols['1']").alias("end"),
                      F.expr("cols['3']['sentence']").alias("sent_id"),
                      F.expr("cols['2']").alias("ner_chunk"),
                      F.expr("cols['3']['entity']").alias("entity"),
                      F.expr("cols['4']").alias("snomed_code"),
                      F.expr("cols['5']['all_k_resolutions']").alias("snomed_texts"),
                      ).toPandas()

In [42]:
df

,begin,end,sent_id,ner_chunk,entity,snomed_code,snomed_texts
0,37,65,0,diabetes mellitus gestacional,clinical_condition,11687002,diabetes mellitus gestacional [diabetes mellitus gestacional]:::diabetes mellitus gestacional que complica el embarazo [diabetes mellitus gestacional que complica el embarazo]:::prediabetes gestacional [prediabetes gestacional]:::antecedente materno de diabetes gestacional [antecedente materno de diabetes gestacional]:::diabetes mellitus gestacional en el momento del nacimiento [diabetes mellitus gestacional en el momento del nacimiento]:::alteración de la tolerancia a la glucosa en el embarazo [alteración de la tolerancia a la glucosa en el embarazo]:::antecedente de embarazo con diabetes mellitus gestacional [antecedente de embarazo con diabetes mellitus gestacional]:::trastorno hiperglucémico del embarazo [trastorno hiperglucémico del embarazo]:::diabetes mellitus gestacional posparto [diabetes mellitus gestacional posparto]:::diabetes mellitus preexistente en el embarazo [diabetes mellitus preexistente en el embarazo]:::embarazo con prueba de tolerancia anormal a la glucosa [embarazo con prueba de tolerancia anormal a la glucosa]
1,128,144,0,diabetes mellitus,clinical_condition,73211009,diabetes mellitus [diabetes mellitus]:::tipo de diabetes [tipo de diabetes]:::diabetología [diabetología]:::trastorno asociado con diabetes mellitus [trastorno asociado con diabetes mellitus]:::diabetes mellitus atípica (trastorno) [diabetes mellitus atípica]:::diabetes mellitus secundaria [diabetes mellitus secundaria]:::diabetes mellitus tipo 2 [diabetes mellitus tipo 2]:::disglucemia (trastorno) [disglucemia]:::insulinopatía (trastorno) [insulinopatía]:::diabetes mellitus tipo I [diabetes mellitus tipo I]:::diabetes lábil [diabetes lábil]:::trastorno de la regulación de la glucosa (trastorno) [trastorno de la regulación de la glucosa]:::diabético que utiliza insulina [diabético que utiliza insulina]
2,184,195,0,pancreatitis,clinical_condition,75694006,pancreatitis [pancreatitis]:::pancreatitis infecciosa (trastorno) [pancreatitis infecciosa]:::enfermedad del páncreas [enfermedad del páncreas]:::dolor pancreático [dolor pancreático]:::pancreatitis metabólica [pancreatitis metabólica]:::pancreatitis crónica reagudizada [pancreatitis crónica reagudizada]:::antecedente de pancreatitis [antecedente de pancreatitis]:::pancreatitis supurativa (trastorno) [pancreatitis supurativa]:::trastorno no diabético del páncreas (trastorno) [trastorno no diabético del páncreas]:::pancreatitis recurrente [pancreatitis recurrente]:::pancreatitis intersticial (trastorno) [pancreatitis intersticial]:::pancreatitis subaguda [pancreatitis subaguda]:::pancreatitis obstructiva [pancreatitis obstructiva]
3,267,281,0,hepatitis aguda,clinical_condition,37871000,"hepatitis aguda [hepatitis aguda]:::hepatitis viral aguda (trastorno) [hepatitis viral aguda]:::hepatitis aguda causada por infección (trastorno) [hepatitis aguda causada por infección]:::hepatitis infecciosa [hepatitis infecciosa]:::insuficiencia hepática aguda causada por virus de hepatitis (trastorno) [insuficiencia hepática aguda causada por virus de hepatitis]:::hepatitis vírica [hepatitis vírica]:::hepatitis aguda - no infecciosa [hepatitis aguda - no infecciosa]:::hepatitis viral aguda fulminante [hepatitis viral aguda fulminante]:::hepatopatía inflamatoria [hepatopatía inflamatoria]:::hepatitis viral aguda tipo A (trastorno) [hepatitis viral aguda tipo A]:::hepatitis, notificación [hepatitis, notificación]:::hepatitis fulminante [hepatitis fulminante]"
4,286,293,0,obesidad,clinical_condition,414916001,obesidad [obesidad]:::obeso [obeso]:::constitución obesa [constitución obesa]:::obesidad según factores contribuyentes [obesidad según factores contribuyentes]:::enfermedad relacionada con la obesidad [enfermedad relacionada con la obesidad]:::obesidad mórbida [obesidad mórbida]:::sobrepeso [sobrepeso]:::obesidad exógena [obesidad exógena]:::obesidad hiperplásica [obesidad hipe